In [1]:
from pandas import read_csv, DataFrame
import pandas as pd
import os
import re #Both patterns and strings to be searched can be Unicode strings as well as 8-bit strings.
import math
import cobra
import cobra.test
from __future__ import print_function
from os.path import join
from cobra.io import write_sbml_model
from cobra.flux_analysis import flux_variability_analysis
#from cobra.medium import minimal_medium
#from cobra.medium import minimal_medium
from IPython.core.interactiveshell import InteractiveShell
import itertools
import re
import memote

InteractiveShell.ast_node_interactivity = "all"

data_dir = "/Users/lizrad/Dev/iVnat"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
model=cobra.io.read_sbml_model(join(data_dir, "iVnat.xml"))
#model=cobra.io.read_legacy_sbml("/Users/lizrad/Documents/Vibrio_folder/iVnat_gapfill_glu.xml")
model

files found: 
iVnat.xml


Name,GCF_001456255_1_rast_metabolic_model
Memory address,0x01127de7b8
Number of metabolites,1842
Number of reactions,1906
Objective expression,-1.0*bio1_reverse_b18f7 + 1.0*bio1
Compartments,"Cytosol_0, Extracellular_0"


## Memote biomass and consistency functions

In [2]:
from memote.support.biomass import find_blocked_biomass_precursors, sum_biomass_weight, find_biomass_precursors, find_direct_metabolites, gam_in_biomass, detect_false_positive_direct_metabolites, bundle_biomass_components, essential_precursors_not_in_biomass

In [3]:
from memote.support.consistency import check_stoichiometric_consistency
from memote.support.helpers import find_compartment_id_in_model

### Finding biomass precursors

In [ ]:
bio_precursor=find_biomass_precursors(model, model.reactions.bio1)

print("Number of found biomass precursors:" , len(bio_precursor))
bio_precursor

### Missing essential precursors

In [ ]:
missing_essential=essential_precursors_not_in_biomass(model, model.reactions.bio1)
missing_essential

len(missing_essential)
for_real=[]
for bio in model.reactions.bio1.metabolites:
    if bio.annotation["metanetx.chemical"] in missing_essential:
        for_real.append(bio.annotation["metanetx.chemical"])
        print(bio.annotation["metanetx.chemical"])
        bio.compartment
    
        
len(for_real)


main_comp = find_compartment_id_in_model(model, 'c')
main_comp


#memote looks for ids not in the annotation field, my IDs contain c0 

In [ ]:

list_missing_met=[]        
for met in model.metabolites:
    for mis in missing_essential_precursors:
        if mis in met.annotation.values():
           
            list_missing_met.append(met.id)
            print(met.id, mis)
            
len(list_missing_met)       
#37 vs. 52 missing metabolite--> duplication in e0/c0



In [ ]:
both=set()      #because of duplicate ids c0 e0  
for met in model.metabolites:
    for mis in missing_essential_precursors:
        if mis in met.annotation.values():
            both.add(mis)
            
len(both)            
print(both)            
  
#no missing essential precursors

### Blocked biomass precursors

In [ ]:
#blocked=find_blocked_biomass_precursors(model, model.reactions.bio1)

blocked_bio=find_blocked_biomass_precursors(model.reactions.bio1, model)
blocked_bio

### Sum biomass weight

In [ ]:
with model:
    
    for met in model.metabolites:
        
        met.formula=re.sub("(([R])\d+)", "", met.formula)
        met.formula=re.sub("([R])" , "", met.formula)
        met.formula=re.sub("(MISSING)" , "", met.formula)
        
    #sum_bio=sum_biomass_weight(model.reactions.bio1)
    #sum_bio
       
            

In [ ]:
sum_bio=sum_biomass_weight(model.reactions.bio1)
sum_bio
       

### Growth associated maintenance

In [ ]:
#growth-associated maintenance
#True if the biomass reaction includes ATP and H2O as reactants and ADP,Pi and H as products, False otherwise.
gam=gam_in_biomass(model, model.reactions.bio1)
gam
model.reactions.bio1.reaction
#What does it check(id?)?

model.metabolites.MNXM2214_c0
model.metabolites.MNXM5258_c0
model.metabolites.MNXM90619_c0



### False positive direct metabolites

### Bundle biomass components

In [ ]:
bundle=bundle_biomass_components(model, model.reactions.bio1)
bundle

### Find direct metabolites

In [4]:

with model: 
    #for met in model.metabolites:
     #   if "(" in met.formula:
      #      print(met.formula, met.id)
            
  
    model.metabolites.MNXM8231_c0.formula="C21H33N5O9P2"
   
    model.metabolites.get_by_id("DNA-6-O-Methyl-Guanines_c0").formula="C10H16N2NO2O6S"
    model.metabolites.MNXM8231_c0.formula
    model.metabolites.get_by_id("DNA-6-O-Methyl-Guanines_c0").formula
    direct_met=find_direct_metabolites(model, model.reactions.bio1, tolerance=1e-06)
    
    direct_met
   
   #     met.formula=re.sub("(([R])\d+)", "", met.formula)
   #     met.formula=re.sub("(([R]))" , "", met.formula)
   #     met.formula=re.sub("(MISSING)" , "", met.formula)
        
    sum_bio=sum_biomass_weight(model.reactions.bio1)
    sum_bio
    

'C21H33N5O9P2'

'C10H16N2NO2O6S'

[<Metabolite MNXM81014_c0 at 0x1153ac160>,
 <Metabolite MNXM51417_c0 at 0x115156470>,
 <Metabolite MG+2_c0 at 0x1154c4550>,
 <Metabolite MN+2_c0 at 0x1156208d0>,
 <Metabolite CU+2_c0 at 0x11527b9b0>,
 <Metabolite ZN+2_c0 at 0x115393a58>,
 <Metabolite K+_c0 at 0x1155c6c18>,
 <Metabolite MNXM78334_c0 at 0x115450da0>,
 <Metabolite CA+2_c0 at 0x115620f28>]

0.9190716038475946

### Stoichiometric consistency

In [ ]:
stoich=check_stoichiometric_consistency(model)
stoich

### Memote repair

from memote import test_model

In [ ]:
#_, result = test_model(model, results=True)

In [ ]:
#result

### Reset the exchange fluxes to zero

Change the exchange fluxes to zero. Then add the ones that are necessary for minimal media. (What if there is no media that we want? How does the media work ? Where does it show? 
Create a dictionary where there is a readable format for the medium. Is there an annotation field for medium? 

In [ ]:
#model.reactions.EX_cpd00009_e0.bounds = (0,1000)

    
with model:
    
    for reac in model.reactions:
        if reac.id.startswith("EX") or  ["MNXR137716_c0", 'MNXR137717_c0', 'MNXR137715_c0']:
            reac.bounds=(0,0)
        
    model.medium 
    model.reactions.EX_cpd00012_e0.bounds = (-1000,1000) #ppi
    model.reactions.EX_cpd00009_e0.bounds = (-1000,1000) #pi
    model.reactions.EX_cpd00209_e0.bounds = (-1000,1000) #NO3
    model.reactions.EX_cpd00075_e0.bounds = (-1000,1000) #NO2
    model.reactions.EX_cpd00013_e0.bounds = (-1000,1000) #NH4
    model.reactions.EX_cpd00048_e0.bounds = (-1000,1000) #SO4
    model.reactions.EX_cpd00027_e0.bounds = (-1000,1000) #D-glucose
    model.reactions.EX_cpd00971_e0.bounds = (-1000,1000) #Na+
    model.reactions.EX_cpd00076_e0.bounds = (-1000,1000) #Sucrose
    model.reactions.EX_cpd00080_e0.bounds = (-1000,1000) #D-Glycerol 1-phosphate
    model.reactions.EX_cpd00001_e0.bounds = (-1000,1000) #H2O
    model.reactions.EX_cpd00011_e0.bounds = (-1000,1000) #CO2
    model.reactions.EX_cpd00067_e0.bounds = (-1000,1000) #H+
    model.reactions.EX_cpd00063_e0.bounds = (-1000,1000) #Ca2+
    model.reactions.EX_cpd00099_e0.bounds = (-1000,1000) #Cl-
    model.reactions.EX_cpd00205_e0.bounds = (-1000,1000) #K+
    model.reactions.EX_cpd00254_e0.bounds = (-1000,1000) #Mg2+
    model.reactions.EX_cpd00030_e0.bounds = (-1000,1000) #Mn2+
    model.reactions.EX_cpd00007_e0.bounds = (-1000,1000) #O2
    model.reactions.MNXR137716_c0.bounds = (-1000,1000) #
    model.reactions.MNXR137715_c0.bounds = (-1000,1000) #
    model.reactions.MNXR137717_c0.bounds = (-1000,1000) #
    model.reactions.EX_cpd00058_e0.bounds = (-1000,1000) #Cu2+
    model.reactions.EX_cpd10515_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd10516_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00034_e0.bounds = (-1000,1000) #Fe2+
    
    model.reactions.EX_cpd00035_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00039_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00041_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd11580_e0.bounds = (-1000,1000) #Fe2+
    model.reactions.EX_cpd00023_e0.bounds = (-1000,1000) #Fe2
    model.slim_optimize()
    

    model.medium
    #model.optimize()
    
    model.reactions.MNXR137715_c0

### Existing media and their description to dict

In [ ]:
chem_xref = read_csv("/Users/lizrad/Documents/Vibrio_folder/chem_xref.tsv" , sep='\t' , low_memory=False)
chem_xref
#499091 530559

df_seed = chem_xref.loc[499088:530556,'XREF':'Description']
pivoted_seed = df_seed.pivot_table(index='XREF',columns='XREF_ID',values='Description',aggfunc = lambda x: x)

#pivoted_seedd=pivoted_seed.drop(['metanetx'])
pivoted_seed
chem_xref_MNX_seed_dict = pivoted_seed.to_dict('index')
chem_xref_MNX_seed_dict2 = chem_xref_MNX_seed_dict['seed']
print(chem_xref_MNX_seed_dict2)


In [ ]:
#MNXR137716_c0','MNXR137717_c0','MNXR137715_c0' dont have seed IDs
medium_dict={}
for med in model.medium:
    if med.startswith("MNXR"):
        continue
    pre, seed, compartment=med.split("_")
    print(seed)
    for k in chem_xref_MNX_seed_dict2.keys():
        if seed in k: 
            print (chem_xref_MNX_seed_dict2[k])
            medium_dict[pre+"_"+seed+"_"+compartment]=chem_xref_MNX_seed_dict2[k]
            
print(medium_dict)

In [ ]:
#check every combinations of mediums :(  
#for L in range(0, len(model.medium)+1):
#    for subset in itertools.combinations(model.medium, L):
#        print(subset)

From KBase download medium in tsv. Try LB(Lysogeny Broth) and minimal nms(nitrate minimal salts medium). With Edit Media Application in KBase one can modify the existing medium) 

Result: No growth (neither).
Check for blocked precursors: both blocked with 76 metabolites. 


In [5]:
media_nms = read_csv("/Users/lizrad/Documents/Vibrio_folder/LB.tsv" , sep='\t' , low_memory=False)
#media_nms = read_csv("/Users/lizrad/Documents/Vibrio_folder/minimal_nms.tsv" , sep='\t' , low_memory=False)
media_nms

comp_list=media_nms["compounds"].tolist()

with model:
    for met in model.metabolites: 
        if met.annotation["seed.compound"] in comp_list: 
            print(met.id, met.annotation["seed.compound"])

        
    #Adding the direct metabolites as exchange reactions
         
    for reac in model.reactions:
        if reac.id.startswith("EX"):
            
            reac.bounds=(0,0)
            reac.id
            pre, seed, compart =reac.id.split("_")
            if seed in comp_list: 
                reac.bounds=(-1000,1000)
            for i in direct_met:
                meta, compart =i.id.split("_")
                if meta in seed:
                    reac.bounds
                    reac.bounds=(-1000,1000)
                    reac.bounds
    for d in direct_met:
        model.add_boundary(model.metabolites.get_by_id(d.id), type="exchange", reaction_id=None, lb=-1000.0, ub=1000.0)
    #
    model.medium
    len(model.medium)
    model.optimize().objective_value
    model.summary()
    
   # blocked_precursors=find_blocked_biomass_precursors(model.reactions.bio1, model)
   # blocked_precursors
   # len(blocked_precursors)
#model.reactions.get_by_id['EX_cpd00067_e0'].bounds

    

,compounds,name,formula,minFlux,maxFlux,concentration
0,cpd00001,H2O,H2O,-100,100,0.001
1,cpd00007,O2,O2,-100,100,0.001
2,cpd00009,Phosphate,HO4P,-100,100,0.001
3,cpd00018,AMP,C10H13N5O7P,-100,100,0.001
4,cpd00023,L-Glutamate,C5H8NO4,-100,100,0.001
5,cpd00027,D-Glucose,C6H12O6,-100,100,0.001
6,cpd00028,Heme,C34H30FeN4O4,-100,100,0.001
7,cpd00030,Mn2+,Mn,-100,100,0.001
8,cpd00033,Glycine,C2H5NO2,-100,100,0.001
9,cpd00034,Zn2+,Zn,-100,100,0.001


PROTON_c0 cpd00067
GLY_c0 cpd00033
CPD-16459_c0 cpd00009
HYPOXANTHINE_c0 cpd00226
CPD-15815_c0 cpd00001
L-ASPARTATE_c0 cpd00041
MET_c0 cpd00060
PROTON_e0 cpd00067
ADENOSINE_e0 cpd00182
ADENOSINE_c0 cpd00182
SER_c0 cpd00054
CMP_c0 cpd00046
AMP_c0 cpd00018
GLT_c0 cpd00023
CYS_c0 cpd00084
Glucopyranose_c0 cpd00027
OXYGEN-MOLECULE_c0 cpd00007
LYS_c0 cpd00039
INOSINE_c0 cpd00246
HSO4_c0 cpd00048
DEOXYCYTIDINE_c0 cpd00654
GUANOSINE_c0 cpd00311
UMP_c0 cpd00091
THIAMINE-P_c0 cpd00793
URIDINE_c0 cpd00249
URACIL_c0 cpd00092
PHE_c0 cpd00066
PRO_e0 cpd00129
CPD0-2422_e0 cpd00971
PRO_c0 cpd00129
CPD0-2422_c0 cpd00971
HIS_c0 cpd00119
MNXM2646_c0 cpd01012
MNXM2646_e0 cpd01012
CPD-7046_c0 cpd00239
SHIKIMATE_c0 cpd00383
PREPHENATE_c0 cpd00219
L-ALPHA-ALANINE_c0 cpd00035
FE+2_c0 cpd10515
FE+3_c0 cpd10516
INOSINE_e0 cpd00246
GMP_c0 cpd00126
CPD-15815_e0 cpd00001
GLT_e0 cpd00023
CL-_c0 cpd00099
CU+2_e0 cpd00058
CU+2_c0 cpd00058
ARG_c0 cpd00051
RIBOFLAVIN_c0 cpd00220
THR_c0 cpd00161
LEU_e0 cpd00107
LEU_c0 

'EX_cpd00067_e0'

'EX_cpd00182_e0'

'EX_cpd00106_e0'

...

Reaction identifier,EX_CA+2_c0
Name,Ca2_plus_ exchange
Memory address,0x011379c390
Stoichiometry,CA+2_c0 <=> Ca2_plus_ <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


{'EX_cpd00067_e0': 1000,
 'EX_cpd00182_e0': 1000,
 'EX_cpd00129_e0': 1000,
 'EX_cpd00971_e0': 1000,
 'EX_cpd01012_e0': 1000,
 'EX_cpd00246_e0': 1000,
 'EX_cpd00001_e0': 1000,
 'EX_cpd00023_e0': 1000,
 'EX_cpd00058_e0': 1000,
 'EX_cpd00107_e0': 1000,
 'EX_cpd00184_e0': 1000,
 'EX_cpd00051_e0': 1000,
 'EX_cpd10515_e0': 1000,
 'EX_cpd00009_e0': 1000,
 'EX_cpd00060_e0': 1000,
 'EX_cpd00438_e0': 1000,
 'EX_cpd00034_e0': 1000,
 'EX_cpd00249_e0': 1000,
 'EX_cpd00119_e0': 1000,
 'EX_cpd00654_e0': 1000,
 'EX_cpd00322_e0': 1000,
 'EX_cpd00054_e0': 1000,
 'EX_cpd00644_e0': 1000,
 'EX_cpd00041_e0': 1000,
 'EX_cpd00035_e0': 1000,
 'EX_cpd00039_e0': 1000,
 'EX_cpd10516_e0': 1000,
 'EX_cpd00254_e0': 1000,
 'EX_cpd00065_e0': 1000,
 'EX_cpd00048_e0': 1000,
 'EX_cpd00149_e0': 1000,
 'EX_cpd00531_e0': 1000,
 'EX_cpd00027_e0': 1000,
 'EX_cpd00226_e0': 1000,
 'EX_cpd00069_e0': 1000,
 'EX_cpd00205_e0': 1000,
 'EX_cpd00007_e0': 1000,
 'EX_cpd03424_e0': 1000,
 'EX_cpd00156_e0': 1000,
 'EX_cpd00092_e0': 1000,


43

1.0051609738699301e-13

IN FLUXES                 OUT FLUXES                OBJECTIVES
------------------------  ------------------------  --------------
ADENOSINE_e0       1e+03  HYPOXANTHINE_e0    1e+03  bio1  1.01e-13
GLT_e0             1e+03  PRO_e0             1e+03
DEOXYCYTIDIN...  685      URACIL_e0        685
PROTON_e0        373      CPD-15815_e0     418
                          URIDINE_e0       369
                          L-ASPARTATE_e0   365
                          L-ALPHA-ALAN...  320
                          PANTOTHENATE_e0  262
                          Glucopyranos...   54.6


In [6]:
direct_list=[]
with model:
    
    for d in direct_met:
        metanet,suff =d.id.split("_")
        direct_list.append(d.id)
        
direct_list



['MNXM81014_c0',
 'MNXM51417_c0',
 'MG+2_c0',
 'MN+2_c0',
 'CU+2_c0',
 'ZN+2_c0',
 'K+_c0',
 'MNXM78334_c0',
 'CA+2_c0']

In [7]:
direct_met_reaction_dict={}
for reac in model.reactions:
    for rmet in reac.metabolites:
    
        if  rmet in direct_met:
            #direct_met_reaction_dict[direct_met].append(rmet)
            print(rmet, "is in the reaction of ", reac)
            
            


MNXM51417_c0 is in the reaction of  MNXR137716_c0:  --> MNXM51417_c0
CU+2_c0 is in the reaction of  MNXR96952_c0: ATP_c0 + CPD-15815_c0 + CU+2_e0 --> ADP_c0 + CPD-16459_c0 + CU+2_c0 + PROTON_c0
ZN+2_c0 is in the reaction of  ABC-63-RXN_c0: ATP_c0 + CPD-15815_c0 + ZN+2_e0 --> ADP_c0 + CPD-16459_c0 + PROTON_c0 + ZN+2_c0
MNXM81014_c0 is in the reaction of  MNXR137717_c0:  --> MNXM81014_c0
MNXM78334_c0 is in the reaction of  MNXR137715_c0:  --> MNXM78334_c0
MG+2_c0 is in the reaction of  3.6.3.2-RXN_c0: ATP_c0 + CPD-15815_c0 + MG+2_e0 --> ADP_c0 + CPD-16459_c0 + MG+2_c0 + PROTON_c0
CU+2_c0 is in the reaction of  3.6.3.4-RXN_c0: ATP_c0 + CPD-15815_c0 + CU+2_c0 --> ADP_c0 + CPD-16459_c0 + CU+2_e0 + PROTON_c0
MG+2_c0 is in the reaction of  TRANS-RXN-141_c0: MG+2_c0 <=> MG+2_e0
K+_c0 is in the reaction of  TRANS-RXN-42_c0: K+_c0 + PROTON_e0 <=> K+_e0 + PROTON_c0
K+_c0 is in the reaction of  TRANS-RXN-143_c0: K+_e0 <=> K+_c0
CA+2_c0 is in the reaction of  MNXR96437_c0: CA+2_e0 + PROTON_c0 --> C

Setting the direct precursors to medium, those that are already in the medium are found, the other direct metabolites cannot be found among the mediums. 

In [16]:
 
with model:
    
    for d in direct_list:
        pre1, suff1 =d.split("_")
        
        
        #create a new metabolite in the e0 compartment
        mcopy=model.metabolites.get_by_id("K+_c0").copy()
        mcopy.compartment = 'e0'
        mcopy.id = pre1+"_e0"
        mcopy.formula=""
        mcopy.name=""
        model.add_metabolites(mcopy)
        
        mcopy
        #create a new reaction between the c0 and e0 compartment
        
        rcopy=model.reactions.get_by_id("TRANS-RXN-143_c0").copy()
        rcopy.id="TRANS_"+d
        for r in rcopy.metabolites:
            r.remove_from_model
        #model.reactions.get_by_id(rcopy).subtract_metabolites()   
        rcopy.name=""
        rcopy.gene_reaction_rule=""
        pre, suff=mcopy.id.split("_")
        rcopy.add_metabolites({
        model.metabolites.get_by_id(pre+"_c0"): -1.0, model.metabolites.get_by_id(mcopy.id): 1.0 })
        rcopy.reaction
        
model.reactions.get_by_id("TRANS-RXN-143_c0")   


Metabolite identifier,MNXM81014_e0
Name,
Memory address,0x011353ec88
Formula,
Compartment,e0
In 0 reaction(s),


<bound method Metabolite.remove_from_model of <Metabolite K+_e0 at 0x11353e518>>

<bound method Metabolite.remove_from_model of <Metabolite K+_c0 at 0x11353e438>>

'K+_e0 + MNXM81014_c0 <=> K+_c0 + MNXM81014_e0'

Metabolite identifier,MNXM51417_e0
Name,
Memory address,0x011353e240
Formula,
Compartment,e0
In 0 reaction(s),


...

Metabolite identifier,CA+2_e0
Name,
Memory address,0x0113556630
Formula,
Compartment,e0
In 0 reaction(s),


<bound method Metabolite.remove_from_model of <Metabolite K+_e0 at 0x1134f9cf8>>

<bound method Metabolite.remove_from_model of <Metabolite K+_c0 at 0x1134f9c18>>

'CA+2_c0 + K+_e0 <=> CA+2_e0 + K+_c0'

Reaction identifier,TRANS-RXN-143_c0
Name,TRANS_RXN_187ce_c0
Memory address,0x01167e2748
Stoichiometry,K+_e0 <=> K+_c0 K_plus_ <=> K_plus_
GPR,PN96_RS22240 or PN96_RS19215
Lower bound,-1000.0
Upper bound,1000.0


In [ ]:
from memote.support.helpers import find_transport_reactions

In [ ]:
for rxn in model.metabolites.MNXM5129_c0.reactions:
    rxn
    if rxn in find_transport_reactions(model):
        print(rxn.id)

In [ ]:
for met in model.reactions.MNXR136333_c0.metabolites:
    print(met.compartment, met.formula)
model.reactions.MNXR136333_c0.copy

In [ ]:
for met in model.reactions.MNXR136261_c0.metabolites:
    print(met, met.compartment, met.formula, met.annotation)
model.reactions.MNXR136261_c0.reaction
model.reactions.MNXR136261_c0.annotation

In [ ]:
for met in model.reactions.MNXR136333_c0.metabolites:
    print(met, met.compartment, met.formula)
model.reactions.MNXR136333_c0.reaction

In [ ]:
len(find_transport_reactions(model))


In [ ]:

with model: 
   # add_ex = model.add_boundary(model.metabolites.MNXM6175_c0, type="exchange")
    #add_ex.id
    
    for d in direct_met:
        if d not in model.medium:
            metd=d.id
            model.metabolites.get_by_id(metd)
            model.add_boundary(model.metabolites.get_by_id(d.id), type="exchange", reaction_id="EX"+"_"+metd, lb=-1000, ub=1000)
        #model.reactions.add(add_ex)
        #model.reactions.EX
    for reac in model.reactions:
        
        if reac.id.startswith("EX"):
            print(reac.id)
    
    model.medium
    model.optimize()
    model.exchanges
    for reac in model.reactions:
        B=is_boundary_type(reac, "exchange", "c0")
    
        if B :
            reac.id
    
        

In [ ]:
from memote.support.helpers import find_biomass_reaction
with model:
    model.reactions.bio1.annotation['sbo'] = 'SBO:0000629'
    #model.reactions.bio1.annotation['SBO'] = 'SBO:0000629'
    model.reactions.bio1.annotation
    #model.reactions.bio1.id="biomass"

    biomass_reac=find_biomass_reaction(model)
    biomass_reac

#write_sbml_model(model, "/Users/lizrad/Dev/iVnat/iVnat.xml")
    

In [ ]:
#what is the id for the selected substrates from seed id
substrate=["cpd00076","cpd00027", "cpd00080","cpd00179", "cpd00082", "cpd00036", "cpd00224", "cpd00314", "cpd00020","cpd00029","cpd00154",
           "cpd00208","cpd00138","cpd00158","cpd00363","cpd00116","cpd00047"]

for met in model.metabolites:
    if met.annotation["seed.compound"] in substrate and met.compartment=="c0":
        print(met.id, met.annotation["seed.compound"] )

        
#does the subsrate have exchange reaction        

ex_substrate=[]        
for exchange in medium_dict.keys():
    ex, seed, suff=exchange.split("_")
    if  seed in substrate:
        print(exchange, medium_dict[exchange])
        ex_substrate.append(exchange)




### Transport reaction bug

In [ ]:
from __future__ import absolute_import

import logging
import re
from collections import defaultdict
from operator import itemgetter


import numpy as np
import pandas as pd
from six import iteritems, itervalues
from sympy import expand
from importlib_resources import open_text
from cobra.exceptions import Infeasible
from cobra.medium import find_boundary_types
from pylru import lrudecorator

import memote.utils as utils
import memote.support.data
import memote.utils as utils
import memote.support.data
from memote.support.helpers import find_biomass_reaction

In [ ]:
import memote
import cobra

In [ ]:
from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs
model2 = Model('example_model')

#def abc_pump_annotations(base):
   # """Provide a model with an ABC transport reaction."""
atp = cobra.Metabolite("atp_c", compartment="c")
adp = cobra.Metabolite("adp_c", compartment="c")
h = cobra.Metabolite("h_c", formula="H",compartment="c")
pi = cobra.Metabolite("pi_c", compartment="c")
h2o = cobra.Metabolite("h2o_c", formula="H2O", compartment="c")
aso_c = cobra.Metabolite("aso3_c", compartment="c")
aso_e = cobra.Metabolite("aso3_e", compartment="e")

atp.annotation["biocyc"] = ["META:ATP", "META:CPD0-1634"]
adp.annotation["biocyc"] = ["META:ADP", "META:CPD0-1651"]
h.annotation["biocyc"] = "META:PROTON"
pi.annotation["biocyc"] = ["META:CPD-16459", "META:CPD-9010"]
h2o.annotation["biocyc"] = ["META:CPD-15815", "META:HYDROXYL-GROUP"]
aso_c.annotation["biocyc"] = ["META:CPD0-2040", "META:CPD-763"]
aso_e.annotation["biocyc"] = ["META:CPD0-2040", "META:CPD-763"]

pump = cobra.Reaction("PUMP")
pump.add_metabolites({aso_c: -1, atp: -1, h2o: -1,
                          adp: 1, h: 1, pi: 1, aso_e: 1})
model2.add_reactions([pump])
####################
MNXM11392_c0 = cobra.Metabolite("MNXM11392_c0", formula="C40H78O13P2", compartment="c" )
MNXM5129_c0 = cobra.Metabolite("MNXM5129_c0", formula="C40H78O10P", compartment="c" )
CPD_16459_c0 = cobra.Metabolite("CPD_16459_c0", formula="HO4P", compartment="c" )

#MNXM11392_c0 = cobra.Metabolite("MNXM11392_c0",  compartment="c" )
#MNXM5129_c0 = cobra.Metabolite("MNXM5129_c0",  compartment="c" )
#CPD_16459_c0=cobra.Metabolite("MNXM5129_c0",  compartment="c" )
h2o.annotation['bigg.metabolite']=['h2o', 'oh1']
h2o.annotation["chebi"]=['CHEBI:15377']
h2o.annotation['inchi']=['InChI=1S/H2O/h1H2']
MNXM11392_c0.annotation={'inchi': 'MISSING', 'inchikey': 'MISSING', 'metanetx.chemical': 'MNXM11392', 'seed.compound': 'cpd15716', 'smiles': 'MISSING'}
CPD_16459_c0.annotation={'bigg.metabolite': 'pi', 'biocyc': ['CPD-16459', 'CPD-9010', 'PHOSPHATE-GROUP', 'Pi'], 'chebi': 'CHEBI:43474', 'inchi': 'InChI=1S/H3O4P/c1-5(2,3)4/h(H3,1,2,3,4)/p-2', 'inchikey': 'NBIIXXVUZAFLBC-UHFFFAOYSA-L', 'kegg.compound': ['C00009', 'C13558'], 'metanetx.chemical': 'MNXM9', 'seed.compound': 'cpd00009', 'smiles': 'OP([O-])([O-])=O'}
MNXM5129_c0.annotation={'inchi': 'MISSING', 'inchikey': 'MISSING', 'metanetx.chemical': 'MNXM5129', 'seed.compound': 'cpd15722', 'smiles': 'MISSING'}



MNXR136261_c0=cobra.Reaction("MNXR136261_c0")
MNXR136261_c0.add_metabolites({ MNXM11392_c0: -1, h2o: -1,
                           h: 1, CPD_16459_c0: 1, MNXM5129_c0: 1})

model2.add_reactions([MNXR136261_c0])
#'CPD-15815_c0 + MNXM11392_c0 --> CPD-16459_c0 + MNXM5129_c0 + PROTON_c0'
####################


GLYCEROL_c0 = cobra.Metabolite("GLYCEROL_c0", formula="C3H8O3", compartment="c" )
MNXM6297_c0 = cobra.Metabolite("MNXM6297_c0", formula="C77H148O17P2", compartment="c" )


MNXR136333_c0=cobra.Reaction("MNXR136333_c0", lower_bound=-1000)
MNXR136333_c0.add_metabolites({ MNXM5129_c0: -2, GLYCEROL_c0: 1,
                           MNXM6297_c0: 1})

model2.add_reactions([MNXR136333_c0])




model2
    #return base
print('%i reaction' % len(model2.reactions))
print('%i metabolites' % len(model2.metabolites))
print('%i genes' % len(model2.genes))
pump.reaction
MNXR136261_c0.reaction
MNXR136333_c0.reaction
MNXM5129_c0.annotation
    


In [ ]:
def find_biomass_reaction(model):
    """
    Return a list of the biomass reaction(s) of the model.
    This function identifies possible biomass reactions using two steps:
    1. Return reactions that include the SBO annotation "SBO:0000629" for
    biomass.
    If no reactions can be identifies this way:
    2. Look for the ``buzzwords`` "biomass", "growth" and "bof" in reaction IDs.
    3. Look for metabolite IDs or names that contain the ``buzzword`` "biomass"
    and obtain the set of reactions they are involved in.
    4. Remove boundary reactions from this set.
    5. Return the union of reactions that match the buzzwords and of the
    reactions that metabolites are involved in that match the buzzword.
    Parameters
    ----------
    model : cobra.Model
        The metabolic model under investigation.
    Returns
    -------
    list
        Identified biomass reactions.
    """
    sbo_matches = set([rxn for rxn in model.reactions if
                       rxn.annotation is not None and
                       'SBO' in rxn.annotation and
                       rxn.annotation['SBO'] == 'SBO:0000629'])

    if len(sbo_matches) > 0:
        return list(sbo_matches)

    buzzwords = ['biomass', 'growth', 'bof']

    buzzword_matches = set([rxn for rxn in model.reactions if any(
        string in rxn.id.lower() for string in buzzwords)])

    biomass_met = []
    for met in model.metabolites:
        if met.id.lower().startswith('biomass') or met.name.lower().startswith(
            'biomass'
        ):
            biomass_met.append(met)
    if biomass_met == 1:
        biomass_met_matches = set(
            biomass_met.reactions
        ) - set(model.boundary)
    else:
        biomass_met_matches = set()

    return list(buzzword_matches | biomass_met_matches)


In [ ]:
def is_transport_reaction_formulae(rxn):
    """
    Return boolean if a reaction is a transport reaction (from formulae).
    Parameters
    ----------
    rxn: cobra.Reaction
        The metabolic reaction under investigation.
    """
    # Collecting criteria to classify transporters by.
    rxn_reactants = set([met.formula for met in rxn.reactants])
    rxn_products = set([met.formula for met in rxn.products])
    # Looking for formulas that stay the same on both side of the reaction.
    transported_mets = \
        [formula for formula in rxn_reactants if formula in rxn_products]
    # Collect information on the elemental differences between
    # compartments in the reaction.
    delta_dicts = find_transported_elements(rxn)
    non_zero_array = [v for (k, v) in iteritems(delta_dicts) if v != 0]
    # Excluding reactions such as oxidoreductases where no net
    # transport of Hydrogen is occurring, but rather just an exchange of
    # electrons or charges effecting a change in protonation.
    if set(transported_mets) != set('H') and list(
        delta_dicts.keys()
    ) == ['H']:
        pass
    # All other reactions for which the amount of transported elements is
    # not zero, which are not part of the model's exchange nor
    # biomass reactions, are defined as transport reactions.
    # This includes reactions where the transported metabolite reacts with
    # a carrier molecule.
    elif sum(non_zero_array):
        return True

In [ ]:
def find_transported_elements(rxn):
    """
    Return a dictionary showing the amount of transported elements of a rxn.
    Collects the elements for each metabolite participating in a reaction,
    multiplies the amount by the metabolite's stoichiometry in the reaction and
    bins the result according to the compartment that metabolite is in. This
    produces a dictionary of dictionaries such as this
    ``{'p': {'C': -1, 'H': -4}, c: {'C': 1, 'H': 4}}`` which shows the
    transported entities. This dictionary is then simplified to only include
    the non-zero elements of one single compartment i.e. showing the precise
    elements that are transported.
    Parameters
    ----------
    rxn : cobra.Reaction
        Any cobra.Reaction containing metabolites.
    """
    element_dist = defaultdict()
    # Collecting elements for each metabolite.
    for met in rxn.metabolites:
        if met.compartment not in element_dist:
            # Multiplication by the metabolite stoichiometry.
            element_dist[met.compartment] = \
                {k: v * rxn.metabolites[met]
                 for (k, v) in iteritems(met.elements)}
        else:
            x = {k: v * rxn.metabolites[met] for (k, v) in
                 iteritems(met.elements)}
            y = element_dist[met.compartment]
            element_dist[met.compartment] = \
                {k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y)}
    delta_dict = defaultdict()
    # Simplification of the resulting dictionary of dictionaries.
    for elements in itervalues(element_dist):
        delta_dict.update(elements)
    # Only non-zero values get included in the returned delta-dict.
    delta_dict = {k: abs(v) for (k, v) in iteritems(delta_dict) if v != 0}
    return delta_dict


In [ ]:
def is_transport_reaction_annotations(rxn):
    """
    Return boolean if a reaction is a transport reaction (from annotations).
    Parameters
    ----------
    rxn: cobra.Reaction
        The metabolic reaction under investigation.
    """
    reactants = set([(k, tuple(v)) for met in rxn.reactants
                     for k, v in iteritems(met.annotation)
                     if met.id is not "H" and k is not None and v is not None])
    #print("reactants", reactants)
    products = set([(k, tuple(v)) for met in rxn.products
                    for k, v in iteritems(met.annotation)
                    if met.id is not "H" and k is not None and v is not None])
    #print("products", products)
    # Find intersection between reactant annotations and
    # product annotations to find common metabolites between them,
    # satisfying the requirements for a transport reaction. Reactions such
    # as those involving oxidoreductases (where no net transport of
    # Hydrogen is occurring, but rather just an exchange of electrons or
    # charges effecting a change in protonation) are excluded.
    transported_mets = reactants & products
    #print("transported_mets", transported_mets, len(transported_mets))
    
    if len(transported_mets) > 0:
        return True


In [ ]:
def find_transport_reactions(model):
    """
    Return a list of all transport reactions.
    Parameters
    ----------
    model : cobra.Model
        The metabolic model under investigation.
    Notes
    -----
    A transport reaction is defined as follows:
    1. It contains metabolites from at least 2 compartments and
    2. at least 1 metabolite undergoes no chemical reaction, i.e.,
    the formula and/or annotation stays the same on both sides of the equation.
    A notable exception is transport via PTS, which also contains the following
    restriction:
    3. The transported metabolite(s) are transported into a compartment through
    the exchange of a phosphate group.
    An example of transport via PTS would be
    pep(c) + glucose(e) -> glucose-6-phosphate(c) + pyr(c)
    Reactions similar to transport via PTS (referred to as "modified transport
    reactions") follow a similar pattern:
    A(x) + B-R(y) -> A-R(y) + B(y)
    Such modified transport reactions can be detected, but only when a formula
    field exists for all metabolites in a particular reaction. If this is not
    the case, transport reactions are identified through annotations, which
    cannot detect modified transport reactions.
    """
    transport_reactions = []
    transport_rxn_candidates = set(model.reactions) - set(model.boundary) \
        - set(find_biomass_reaction(model))
    #print("transport_rxn_candidates",transport_rxn_candidates)
    # Add all labeled transport reactions
    sbo_matches = set([rxn for rxn in transport_rxn_candidates if
                       rxn.annotation is not None and
                       'SBO' in rxn.annotation and
                       rxn.annotation['SBO'] in TRANSPORT_RXN_SBO_TERMS])
    if len(sbo_matches) > 0:
        transport_reactions += list(sbo_matches)
    # Find unlabeled transport reactions via formula or annotation checks
    for rxn in transport_rxn_candidates:
        # Check if metabolites have formula field
        rxn_mets = set([met.formula for met in rxn.metabolites])
        if (None not in rxn_mets) and (len(rxn_mets) != 0):
            if is_transport_reaction_formulae(rxn):
                transport_reactions.append(rxn)
        elif is_transport_reaction_annotations(rxn): #
            transport_reactions.append(rxn)
       # print("rxn_mets",rxn_mets)
    return set(transport_reactions)

    

In [ ]:
for rxn in model.metabolites.MNXM5129_c0.reactions:
    
    if rxn in find_transport_reactions(model):
        print(rxn.id)

In [ ]:
model2_trans=memote.support.helpers.find_transport_reactions(model2)
model2_trans
#find_direct_metabolites(model, bio1)

In [ ]:
model2_bio=find_biomass_reaction(model2)
model2_bio

In [ ]:
find_transported_elements(MNXR136261_c0)

In [ ]:
istranspann=is_transport_reaction_annotations(MNXR136261_c0)
istranspann

In [ ]:

from memote.support.helpers import find_compartment_id_in_model, find_met_in_model

ESSENTIAL_PRECURSOR_IDS = \
    ['MNXM94', 'MNXM55', 'MNXM134', 'MNXM76', 'MNXM61',
     'MNXM97', 'MNXM53', 'MNXM114', 'MNXM42', 'MNXM142',
     'MNXM37', 'MNXM89557', 'MNXM231', 'MNXM70', 'MNXM78',
     'MNXM199', 'MNXM140', 'MNXM32', 'MNXM29', 'MNXM147',
     'MNXM286', 'MNXM360', 'MNXM394', 'MNXM344',
     'MNXM3', 'MNXM51', 'MNXM63', 'MNXM121',
     'MNXM8', 'MNXM5', 'MNXM16', 'MNXM33', 'MNXM161',
     'MNXM12', 'MNXM256', 'MNXM119', 'MNXM2']

def essential_precursors_not_in_biomass(model, reaction):
   
   
    main_comp =find_compartment_id_in_model(model, 'c')
    biomass_eq = bundle_biomass_components(model, reaction)
    pooled_precursors = set(
        [met for rxn in biomass_eq for met in rxn.metabolites])
    print(pooled_precursors)
    missing_essential_precursors = []
    for mnx_id in ESSENTIAL_PRECURSOR_IDS:
        try:
            met = find_met_in_model(model, mnx_id, main_comp)[0]
            if met not in pooled_precursors:
                missing_essential_precursors.append(met.id)
        except RuntimeError:
            missing_essential_precursors.append(mnx_id)

    return missing_essential_precursors


In [ ]:
missing_essential=essential_precursors_not_in_biomass(model, model.reactions.bio1)
missing_essential
